In [1]:
import pandas as pd
import numpy as np

In [2]:
message=pd.read_csv('C:/Users/harshit/Downloads/spam/SMSSpamCollection',sep='\t',
                   names=['labels','message'])

In [3]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harshit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harshit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps=PorterStemmer()
lt=WordNetLemmatizer()

In [5]:
corpus = []
for i in range(0,len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    

In [6]:
import keras

Using TensorFlow backend.


In [7]:
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM,Dense

In [8]:
voc_size=5000

In [9]:
onehot_repr = [one_hot(words,voc_size)for words in corpus]
onehot_repr

[[3774,
  2198,
  829,
  3628,
  4512,
  1806,
  4254,
  425,
  3768,
  3145,
  115,
  1448,
  4085,
  2358,
  1492,
  2698],
 [555, 2944, 2530, 3307, 688, 1783],
 [3398,
  4132,
  2062,
  1399,
  796,
  4676,
  3599,
  424,
  199,
  1238,
  47,
  2582,
  4676,
  3492,
  4132,
  127,
  3020,
  798,
  4055,
  804,
  613],
 [688, 2766, 4698, 664, 4570, 688, 804, 2617, 4698],
 [4420, 1343, 3571, 2731, 104, 4232, 4564],
 [24,
  2811,
  3874,
  1761,
  2486,
  4653,
  282,
  3388,
  1912,
  1125,
  555,
  241,
  3020,
  4093,
  1554,
  4004],
 [1390, 3621, 282, 2791, 433, 282, 178, 746],
 [2226,
  4512,
  2909,
  2909,
  3884,
  3916,
  4931,
  3023,
  3068,
  2145,
  602,
  2371,
  4070,
  1869,
  2145],
 [3095,
  3979,
  4326,
  3364,
  4272,
  2325,
  4529,
  875,
  1107,
  2151,
  1107,
  1444,
  4377,
  4954,
  1677],
 [2713,
  4721,
  688,
  2028,
  3813,
  529,
  93,
  2336,
  2713,
  4804,
  3398,
  2151,
  2713,
  529,
  181,
  3398],
 [6, 3920, 1490, 4925, 2886, 367, 2542, 347, 41

In [10]:
sent_len= 20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_len)
print(embedded_docs)

[[   0    0    0 ... 2358 1492 2698]
 [   0    0    0 ... 3307  688 1783]
 [4132 2062 1399 ... 4055  804  613]
 ...
 [   0    0    0 ... 3921 4390 2515]
 [   0    0    0 ... 4446 2029 3398]
 [   0    0    0 ...  696 2251  463]]


In [11]:
features = 40
model = Sequential()
model.add(Embedding(voc_size,features,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

W0721 16:25:12.241864 13740 deprecation.py:323] From C:\Users\harshit\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
y=pd.get_dummies(message['labels'])
y=y.iloc[:,1].values


In [13]:
x_final=np.array(embedded_docs)
y_final=np.array(y)

In [15]:
y_final.shape

(5572,)

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size = 0.30, random_state = 0)

In [18]:
x_train.shape

(3900, 20)

In [19]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=390)

W0607 09:51:46.296476  6900 deprecation_wrapper.py:119] From C:\Users\harshit\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 3900 samples, validate on 1672 samples
Epoch 1/10
3900/3900 [==============================] - 10s 3ms/step - loss: 0.6497 - accuracy: 0.8154 - val_loss: 0.5414 - val_accuracy: 0.8678
Epoch 2/10
3900/3900 [==============================] - 9s 2ms/step - loss: 0.3933 - accuracy: 0.8651 - val_loss: 0.2994 - val_accuracy: 0.8678
Epoch 3/10
3900/3900 [==============================] - 10s 2ms/step - loss: 0.2768 - accuracy: 0.8674 - val_loss: 0.2348 - val_accuracy: 0.8750
Epoch 4/10
3900/3900 [==============================] - 6s 2ms/step - loss: 0.2068 - accuracy: 0.9036 - val_loss: 0.1790 - val_accuracy: 0.9396
Epoch 5/10
3900/3900 [==============================] - 7s 2ms/step - loss: 0.1481 - accuracy: 0.9579 - val_loss: 0.1319 - val_accuracy: 0.9707
Epoch 6/10
3900/3900 [==============================] - 7s 2ms/step - loss: 0.1069 - accuracy: 0.9818 - val_loss: 0.1041 - val_accuracy: 0.9767
Epoch 7/10
3900/3900 [==============================] - 8s 2ms/step - loss: 0.0771 - a

In [28]:
score=model.evaluate(x_test,y_test,verbose=0)
print("loss",score[0])
print("accuracy",score[1])

loss 0.056353921783669145
accuracy 0.9850478172302246


In [20]:
pred=model.predict_classes(x_test)

In [25]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,mean_squared_error

In [22]:
accuracy_score(y_test,pred)

0.9850478468899522

In [26]:
mean_squared_error(y_test,pred)

0.014952153110047847

In [23]:
pred1=model.predict_classes(x_train)


In [24]:
accuracy_score(y_train,pred1)

0.9943589743589744

In [27]:
mean_squared_error(y_train,pred1)

0.0056410256410256415